In [1]:
%matplotlib inline

from datetime import datetime
import pyspark
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('default') # haciendo los graficos un poco mas bonitos xD
plt.rcParams['figure.figsize'] = (15, 5)

In [2]:
try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')
    spark = pyspark.sql.SparkSession(sc)
sqlContext = pyspark.SQLContext(sc)

In [3]:
postulantes = pd.read_csv('./fiuba_1_postulantes_educacion.csv')
postulantes = pd.merge(postulantes, pd.read_csv('./fiuba_2_postulantes_genero_y_edad.csv'), on="idpostulante")
postulantes.head()

,idpostulante,nombre,estado,fechanacimiento,sexo
0,NdJl,Posgrado,En Curso,1969-05-09,MASC
1,NdJl,Universitario,Graduado,1969-05-09,MASC
2,8BkL,Universitario,En Curso,1976-07-21,FEM
3,8BkL,Universitario,Graduado,1976-07-21,FEM
4,8BkL,Posgrado,Graduado,1976-07-21,FEM


In [4]:
postulantes.nombre.unique()

array(['Posgrado', 'Universitario', 'Master', 'Otro',
       'Terciario/T\xc3\xa9cnico', 'Secundario', 'Doctorado'],
      dtype=object)

In [5]:
postulantes.estado.unique()

array(['En Curso', 'Graduado', 'Abandonado'], dtype=object)

In [6]:
postulantes.dropna(inplace=True)
postulantes.count()

idpostulante       294668
nombre             294668
estado             294668
fechanacimiento    294668
sexo               294668
dtype: int64

In [7]:
def  fecha_a_anio(fecha):
    anio = fecha.split('-')[0]
    return int(anio) if anio.isdigit() else 0
    
def anio_a_edad(anio):
    return 2018 - anio

In [8]:
postulantes['edad'] = postulantes['fechanacimiento'].apply(fecha_a_anio).apply(anio_a_edad)
postulantes.drop(columns=['fechanacimiento'], inplace=True)
postulantes['edad'].head()

0    49
1    49
2    42
3    42
4    42
Name: edad, dtype: int64

In [9]:
postulantes = postulantes[(postulantes['edad'] >= 18) & (postulantes['edad'] < 65)]

In [10]:
avisos = pd.read_csv('./fiuba_6_avisos_detalle.csv')
print(avisos.columns)
avisos = avisos[['idaviso', 'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area']]
avisos.dropna(inplace=True)
avisos.head(5)

Index([u'idaviso', u'idpais', u'titulo', u'descripcion', u'nombre_zona',
       u'ciudad', u'mapacalle', u'tipo_de_trabajo', u'nivel_laboral',
       u'nombre_area', u'denominacion_empresa'],
      dtype='object')


,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area
0,8725750,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial
1,17903700,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Salud
2,1000150677,Capital Federal,Full-time,Senior / Semi-Senior,Transporte
3,1000610287,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte
4,1000872556,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Producción


In [11]:
avisos.count()

idaviso            13534
nombre_zona        13534
tipo_de_trabajo    13534
nivel_laboral      13534
nombre_area        13534
dtype: int64

In [12]:
vistas = pd.read_csv('./fiuba_3_vistas.csv')
vistas['count'] = 1
vistas['idaviso'] = vistas['idAviso']
vistas.drop(columns=['idAviso'], inplace=True)
vistas.head()

,timestamp,idpostulante,count,idaviso
0,2018-02-23T13:38:13.187-0500,YjVJQ6Z,1,1111780242
1,2018-02-23T13:38:14.296-0500,BmVpYoR,1,1112263876
2,2018-02-23T13:38:14.329-0500,wVkBzZd,1,1112327963
3,2018-02-23T13:38:17.921-0500,OqmP9pv,1,1112318643
4,2018-02-23T13:38:18.973-0500,DrpbXDP,1,1111903673


In [13]:
vistas_por_aviso = vistas.groupby('idaviso')['count'].sum().reset_index()[['idaviso', 'count']]
vistas_por_aviso.head()

,idaviso,count
0,1585950,1
1,1672250,2
2,1799300,1
3,2613010,4
4,3006520,1


In [14]:
postulaciones = pd.read_csv('./fiuba_4_postulaciones.csv')
postulaciones['count'] = 1
postulaciones.head()

,idaviso,idpostulante,fechapostulacion,count
0,1112257047,NM5M,2018-01-15 16:22:34,1
1,1111920714,NM5M,2018-02-06 09:04:50,1
2,1112346945,NM5M,2018-02-22 09:04:47,1
3,1112345547,NM5M,2018-02-22 09:04:59,1
4,1112237522,5awk,2018-01-25 18:55:03,1


In [15]:
postulaciones_por_aviso = postulaciones.groupby('idaviso')['count'].sum().reset_index()[['idaviso', 'count']]
postulaciones_por_aviso.head()

,idaviso,count
0,8725750,117
1,11740890,7
2,12543760,986
3,12812680,115
4,1000132160,11


In [16]:
avisos = avisos.merge(postulaciones_por_aviso, on='idaviso')
avisos.rename(columns={'count':'posts_count'}, inplace=True)
avisos.head()

,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,posts_count
0,8725750,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial,117
1,1000610287,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,182
2,1001135716,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ventas,167
3,1001326344,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ventas,13
4,1001455648,Gran Buenos Aires,Full-time,Junior,Ventas,176


In [17]:
avisos = avisos.merge(vistas_por_aviso, on='idaviso')
avisos.rename(columns={'count':'vistas_count'}, inplace=True)
avisos.head()

,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,posts_count,vistas_count
0,1000610287,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,182,70
1,1001135716,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ventas,167,60
2,1001326344,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ventas,13,37
3,1001455648,Gran Buenos Aires,Full-time,Junior,Ventas,176,47
4,1001724563,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Salud,32,3


In [18]:
avisos.dropna(inplace=True)

In [38]:
posts_por_area = avisos.groupby('nombre_area')['posts_count'].sum().reset_index()
areas_a_borrar = list(posts_por_area[(posts_por_area['posts_count'] < 500)]['nombre_area'].unique())
areas_a_borrar

['Administraci\xc3\xb3n de Base de Datos',
 'Apoderado Aduanal',
 'Bioqu\xc3\xadmica',
 'Comercio Exterior',
 'Comunicaciones Externas',
 'Data Warehousing',
 'Direcci\xc3\xb3n',
 'Dise\xc3\xb1o 3D',
 'Dise\xc3\xb1o Industrial',
 'Dise\xc3\xb1o Multimedia',
 'Dise\xc3\xb1o Textil e Indumentaria',
 'Dise\xc3\xb1o Web',
 'Educaci\xc3\xb3n especial',
 'Emergentolog\xc3\xada',
 'Est\xc3\xa9tica y Cuidado Personal',
 'Exploraci\xc3\xb3n Minera y Petroquimica',
 'Farmacia hospitalaria',
 'Idiomas',
 'Independientes',
 'Ingenier\xc3\xada  Metalurgica',
 'Ingenier\xc3\xada Geol\xc3\xb3gica',
 'Ingenier\xc3\xada en Minas',
 'Ingenier\xc3\xada en Petr\xc3\xb3leo y Petroqu\xc3\xadmica',
 'Instrumentaci\xc3\xb3n',
 'Internet',
 'Investigaci\xc3\xb3n y Desarrollo',
 'Medio Ambiente',
 'Multimedia',
 'Negocios Internacionales',
 'Otras \xc3\xa1reas t\xc3\xa9cnicas en salud',
 'Planeamiento',
 'Qu\xc3\xadmica',
 'Tecnico de Seguros',
 'Telefonista',
 'Traduccion',
 'Veterinaria']

In [39]:
avisos['ctr'] = avisos['posts_count']/avisos['vistas_count']
avisos['ctr'] = avisos['ctr'].apply(lambda x: 1 if x > 1 else x)
avisos.head()

,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,posts_count,vistas_count,ctr
0,1000610287,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,182,70,1.000000
1,1001135716,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ventas,167,60,1.000000
2,1001326344,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ventas,13,37,0.351351
3,1001455648,Gran Buenos Aires,Full-time,Junior,Ventas,176,47,1.000000
4,1001724563,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Salud,32,3,1.000000


In [40]:
avisos.count()

idaviso            6525
nombre_zona        6525
tipo_de_trabajo    6525
nivel_laboral      6525
nombre_area        6525
posts_count        6525
vistas_count       6525
ctr                6525
dtype: int64

In [41]:
postulaciones = postulaciones[['idaviso', 'idpostulante']]
avisos = avisos.merge(postulaciones, on="idaviso")
avisos.head()

,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,posts_count,vistas_count,ctr,idpostulante
0,1000610287,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,182,70,1.0,ZOEEP1
1,1000610287,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,182,70,1.0,53pAYM
2,1000610287,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,182,70,1.0,LNwXl1q
3,1000610287,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,182,70,1.0,wVYPWdV
4,1000610287,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,182,70,1.0,NzdqO9x


In [42]:
avisos.count()

idaviso            2545687
nombre_zona        2545687
tipo_de_trabajo    2545687
nivel_laboral      2545687
nombre_area        2545687
posts_count        2545687
vistas_count       2545687
ctr                2545687
idpostulante       2545687
dtype: int64

In [43]:
avisos = avisos.merge(postulantes, on='idpostulante')
avisos.head()

,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,posts_count,vistas_count,ctr,idpostulante,nombre,estado,sexo,edad
0,1000610287,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,182,70,1.0,53pAYM,Secundario,En Curso,MASC,33
1,1112313739,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Oficios y Profesiones,65,31,1.0,53pAYM,Secundario,En Curso,MASC,33
2,1112342374,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Distribución,210,145,1.0,53pAYM,Secundario,En Curso,MASC,33
3,1000610287,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,182,70,1.0,LNwXl1q,Secundario,Graduado,MASC,25
4,1112214523,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Producción,2625,542,1.0,LNwXl1q,Secundario,Graduado,MASC,25


In [44]:
avisos.count()

idaviso            4107452
nombre_zona        4107452
tipo_de_trabajo    4107452
nivel_laboral      4107452
nombre_area        4107452
posts_count        4107452
vistas_count       4107452
ctr                4107452
idpostulante       4107452
nombre             4107452
estado             4107452
sexo               4107452
edad               4107452
dtype: int64

In [45]:
avisos.drop(columns=['idpostulante'], inplace=True)

In [46]:
avisos.drop(columns=['ctr'], inplace=True)

In [47]:
avisos[['nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'nombre', 'estado', 'sexo', 'nombre_area']] = avisos[['nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'nombre', 'estado', 'sexo', 'nombre_area']].apply(lambda x: x.astype('category'))

In [48]:
avisos.drop(columns=['posts_count', 'vistas_count'], inplace=True)

In [49]:
avisos.head()

,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,nombre,estado,sexo,edad
0,1000610287,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,Secundario,En Curso,MASC,33
1,1112313739,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Oficios y Profesiones,Secundario,En Curso,MASC,33
2,1112342374,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Distribución,Secundario,En Curso,MASC,33
3,1000610287,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,Secundario,Graduado,MASC,25
4,1112214523,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Producción,Secundario,Graduado,MASC,25


In [50]:
avisos.count()

idaviso            4107452
nombre_zona        4107452
tipo_de_trabajo    4107452
nivel_laboral      4107452
nombre_area        4107452
nombre             4107452
estado             4107452
sexo               4107452
edad               4107452
dtype: int64

In [52]:
avisos = avisos[~avisos['nombre_area'].isin(areas_a_borrar)]
avisos.count()

idaviso            4094231
nombre_zona        4094231
tipo_de_trabajo    4094231
nivel_laboral      4094231
nombre_area        4094231
nombre             4094231
estado             4094231
sexo               4094231
edad               4094231
dtype: int64

In [53]:
avisos.head()

,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,nombre,estado,sexo,edad
0,1000610287,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,Secundario,En Curso,MASC,33
1,1112313739,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Oficios y Profesiones,Secundario,En Curso,MASC,33
2,1112342374,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Distribución,Secundario,En Curso,MASC,33
3,1000610287,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,Secundario,Graduado,MASC,25
4,1112214523,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Producción,Secundario,Graduado,MASC,25


In [ ]:
avisos.dtypes

In [54]:
avisos2 = pd.read_csv('./fiuba_6_avisos_detalle.csv')
print(avisos.columns)
avisos2 = avisos2[['idaviso', 'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area']]
avisos[['nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area']] = avisos[['nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area']].apply(lambda x: x.astype('category'))
avisos2.dropna(inplace=True)
vistas.rename(columns={'idAviso':'idaviso'}, inplace=True)
vistas = vistas[['idaviso', 'idpostulante']]
post = pd.DataFrame(postulaciones['idaviso'].unique(), columns=['idaviso'])
avisos2 = avisos2.merge(post, on='idaviso', how='left')
avisos2 = avisos2.merge(vistas, on="idaviso")
avisos2 = avisos2.merge(postulantes, on='idpostulante')
avisos2[['nombre', 'estado', 'sexo']] = avisos2[['nombre', 'estado', 'sexo']].apply(lambda x: x.astype('category'))
avisos2 = avisos2[~avisos2['nombre_area'].isin(areas_a_borrar)]
avisos2.head()

Index([u'idaviso', u'nombre_zona', u'tipo_de_trabajo', u'nivel_laboral',
       u'nombre_area', u'nombre', u'estado', u'sexo', u'edad'],
      dtype='object')


,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,idpostulante,nombre,estado,sexo,edad
0,17903700,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Salud,OqbXzRv,Otro,Graduado,FEM,41
1,17903700,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Salud,OqbXzRv,Terciario/Técnico,Graduado,FEM,41
2,1112352260,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Medicina,OqbXzRv,Otro,Graduado,FEM,41
3,1112352260,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Medicina,OqbXzRv,Terciario/Técnico,Graduado,FEM,41
4,1112352260,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Medicina,OqbXzRv,Otro,Graduado,FEM,41


In [55]:
avisos2.count()

idaviso            1416350
nombre_zona        1416350
tipo_de_trabajo    1416350
nivel_laboral      1416350
nombre_area        1416350
idpostulante       1416350
nombre             1416350
estado             1416350
sexo               1416350
edad               1416350
dtype: int64

In [56]:
avisos2.drop(columns=['idpostulante'], inplace=True)
avisos2.head()

,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,nombre,estado,sexo,edad
0,17903700,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Salud,Otro,Graduado,FEM,41
1,17903700,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Salud,Terciario/Técnico,Graduado,FEM,41
2,1112352260,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Medicina,Otro,Graduado,FEM,41
3,1112352260,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Medicina,Terciario/Técnico,Graduado,FEM,41
4,1112352260,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Medicina,Otro,Graduado,FEM,41


In [57]:
avisos['se_postula'] = 1

In [58]:
avisos2['se_postula'] = 0

In [59]:
avisos.head()

,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,nombre,estado,sexo,edad,se_postula
0,1000610287,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,Secundario,En Curso,MASC,33,1
1,1112313739,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Oficios y Profesiones,Secundario,En Curso,MASC,33,1
2,1112342374,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Distribución,Secundario,En Curso,MASC,33,1
3,1000610287,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Transporte,Secundario,Graduado,MASC,25,1
4,1112214523,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Producción,Secundario,Graduado,MASC,25,1


In [60]:
avisos2.head()

,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,nombre,estado,sexo,edad,se_postula
0,17903700,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Salud,Otro,Graduado,FEM,41,0
1,17903700,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Salud,Terciario/Técnico,Graduado,FEM,41,0
2,1112352260,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Medicina,Otro,Graduado,FEM,41,0
3,1112352260,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Medicina,Terciario/Técnico,Graduado,FEM,41,0
4,1112352260,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Medicina,Otro,Graduado,FEM,41,0


In [61]:
aviso= None
avisos = pd.concat([avisos, avisos2], ignore_index=True)

In [62]:
avisos = pd.get_dummies(avisos)
avisos.head()

,idaviso,edad,se_postula,nombre_zona_Buenos Aires (fuera de GBA),nombre_zona_Capital Federal,nombre_zona_GBA Oeste,nombre_zona_Gran Buenos Aires,tipo_de_trabajo_Fines de Semana,tipo_de_trabajo_Full-time,tipo_de_trabajo_Part-time,...,nombre_Posgrado,nombre_Secundario,nombre_Terciario/Técnico,nombre_Universitario,estado_Abandonado,estado_En Curso,estado_Graduado,sexo_FEM,sexo_MASC,sexo_NO_DECLARA
0,1000610287,33,1,0,0,0,1,0,1,0,...,0,1,0,0,0,1,0,0,1,0
1,1112313739,33,1,0,0,0,1,0,1,0,...,0,1,0,0,0,1,0,0,1,0
2,1112342374,33,1,0,0,0,1,0,1,0,...,0,1,0,0,0,1,0,0,1,0
3,1000610287,25,1,0,0,0,1,0,1,0,...,0,1,0,0,0,0,1,0,1,0
4,1112214523,25,1,0,0,0,1,0,1,0,...,0,1,0,0,0,0,1,0,1,0


In [63]:
avisos.drop(columns=['idaviso'], inplace=True)

In [64]:
edad_min = avisos['edad'].min()
edad_max = avisos['edad'].max()
avisos['edad'] = avisos['edad'].apply(lambda x: (x-edad_min)/float(edad_max-edad_min))
avisos.head()

,edad,se_postula,nombre_zona_Buenos Aires (fuera de GBA),nombre_zona_Capital Federal,nombre_zona_GBA Oeste,nombre_zona_Gran Buenos Aires,tipo_de_trabajo_Fines de Semana,tipo_de_trabajo_Full-time,tipo_de_trabajo_Part-time,tipo_de_trabajo_Pasantia,...,nombre_Posgrado,nombre_Secundario,nombre_Terciario/Técnico,nombre_Universitario,estado_Abandonado,estado_En Curso,estado_Graduado,sexo_FEM,sexo_MASC,sexo_NO_DECLARA
0,0.326087,1,0,0,0,1,0,1,0,0,...,0,1,0,0,0,1,0,0,1,0
1,0.326087,1,0,0,0,1,0,1,0,0,...,0,1,0,0,0,1,0,0,1,0
2,0.326087,1,0,0,0,1,0,1,0,0,...,0,1,0,0,0,1,0,0,1,0
3,0.152174,1,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,1,0,1,0
4,0.152174,1,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,1,0,1,0


In [68]:
import gc
del [[postulantes,postulaciones,avisos2,vistas,post,postulaciones_por_aviso, X, y]]
postulantes = None
postulaciones = None 
avisos2 = None 
vistas = None
post = None
postulaciones_por_aviso = None
X = None
y = None
gc.collect()

0

In [69]:
from scipy.sparse import csr_matrix
y = avisos.se_postula.values
X = csr_matrix(avisos.drop('se_postula', axis=1).as_matrix())
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
print("X_train size: {} | y_train size: {} | X_test size: {} | y_test size: {}".format(X_train.shape, y_train.shape, X_test.shape, y_test.shape))

/home/fabrizio/.local/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


X_train size: (4132935, 155) | y_train size: (4132935,) | X_test size: (1377646, 155) | y_test size: (1377646,)


In [77]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import Perceptron
import time

classifiers = ({'Baseline' :               DummyClassifier(strategy='uniform'), 
                'Decision Tree' :          DecisionTreeClassifier(max_depth=10),
                'Naive Bayes' :            MultinomialNB(),
                'Multi Layer Perceptron' : MLPClassifier(hidden_layer_sizes=30, activation='logistic'),
                'Perceptron' :             Perceptron(penalty='l2')
                })
results = {}

for (clf_label, clf) in classifiers.items():
    t0 = time.time()
    clf.fit(X_train, y_train)
    t1 = time.time()
    predicted = clf.predict(X_test)
    print("Params", clf.get_params())
    print("{} Classifier score on training set: {}".format(clf_label, clf.score(X_train, y_train)))
    print("{} Classifier score on validation set: {}".format(clf_label, clf.score(X_test, y_test)))
    print("{} Classifier correctly predicted: {}".format(clf_label, accuracy_score(y_test, predicted, normalize=True)))
    print("{} F1-score for validation set: {}".format(clf_label, f1_score(y_test, predicted)))
    print("{} Classifier time needed to train: {}".format(clf_label, t1-t0))
    print()

('Params', {'beta_1': 0.9, 'warm_start': False, 'beta_2': 0.999, 'shuffle': True, 'verbose': False, 'nesterovs_momentum': True, 'hidden_layer_sizes': 30, 'epsilon': 1e-08, 'activation': 'logistic', 'max_iter': 200, 'batch_size': 'auto', 'power_t': 0.5, 'random_state': None, 'learning_rate_init': 0.001, 'tol': 0.0001, 'validation_fraction': 0.1, 'alpha': 0.0001, 'solver': 'adam', 'momentum': 0.9, 'learning_rate': 'constant', 'early_stopping': False})
Multi Layer Perceptron Classifier score on training set: 0.747097401725
Multi Layer Perceptron Classifier score on validation set: 0.746760778894
Multi Layer Perceptron Classifier correctly predicted: 0.746760778894
Multi Layer Perceptron F1-score for validation set: 0.852639839966
Multi Layer Perceptron Classifier time needed to train: 261.111381054
()
('Params', {'warm_start': False, 'n_iter': None, 'n_jobs': 1, 'eta0': 1.0, 'verbose': 0, 'shuffle': True, 'fit_intercept': True, 'max_iter': None, 'penalty': 'l2', 'random_state': 0, 'tol': 

156

In [88]:
X_user_0 = X.getrow(0)
X_user_0.todense()

matrix([[0.32608696, 0.        , 0.        , 0.        , 1.        ,
         0.        , 1.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 1.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0

In [85]:
y_user_0 = y.tolist()[0]
y_user_0

1

In [87]:
clf.predict_proba(X_user_0)

array([[0.24203928, 0.75796072]])